### Decision Tree Classifier

In this notebook we are going to implement the Decision Tree Algorithm from scratch using popular python packages such as:

1. numpy
2. Counter
3. sklearn


Decision tree is one of the most powerfull supervised learning classification algorithm. It works for both continuous as well as categorical output variables.

<img src="https://media.geeksforgeeks.org/wp-content/uploads/decisionTree.png" alt="" width="100%"/>

### Assumptions we make while using Decision tree.
* At the beginning, we consider the whole training set as the root.
* Attributes are assumed to be categorical for information gain and for gini index, attributes are assumed to be continuous.
* On the basis of attribute values records are distributed recursively.
* We use statistical methods for ordering attributes as root or internal node.

### Pseudocode
* Find the best attribute and place it on the root node of the tree.
* Now, split the training set of the dataset into subsets. While making the subset make sure that each subset of training dataset should have the same value for an attribute.
* Find leaf nodes in all branches by repeating 1 and 2 on each subset.

![img](https://media.geeksforgeeks.org/wp-content/uploads/decisionTree2.png)

> For more theory about this algorithm please visit the referenced website of geeks for geeks at the end of this notebook.


### Basic imports



In [1]:
from collections import Counter
import numpy as np

### The `entropy` global function
Entropy is the measure of uncertainty of a random variable, it characterizes the impurity of an arbitrary collection of examples. The higher the entropy the more the information content.

![img](https://www.geeksforgeeks.org/wp-content/ql-cache/quicklatex.com-4af22a0bc64a8814e17e83fa12a05eef_l3.svg)


In [2]:
def entropy(y):
  hist = np.bincount(y)
  ps = hist / len(y)
  return -np.sum([p * np.log2(p) for p in ps if p > 0])

### The `Node` helper class

In [3]:
class Node:
  def __init__(
      self, feature=None, threshold=None, left=None, right=None, *, value=None
  ):
    self.feature = feature
    self.threshold = threshold
    self.left = left
    self.right = right
    self.value = value

  def is_leaf_node(self):
    return self.value is not None

### Decision Tree Algorithim

In [9]:
class DecisionTree:
  def __init__(self, min_samples_split=2, max_depth=100, n_feats=None):
    self.min_samples_split = min_samples_split
    self.max_depth = max_depth
    self.n_feats = n_feats
    self.root = None

  def fit(self, X, y):
    """
    In the fit method of a decision tree we will grow the tree
    """
    self.n_feats = X.shape[1] if not self.n_feats else min(self.n_feats, X.shape[1])
    self.root = self._grow_tree(X, y)

  def predict(self, X):
    """
    In the predict method we traverse down the treee
    """
    return np.array([self._traverse_tree(x, self.root) for x in X])

  def _grow_tree(self, X, y, depth=0):
    n_samples, n_features = X.shape
    n_labels = len(np.unique(y))
    # stopping criteria
    if (
        depth >= self.max_depth
        or n_labels == 1
        or n_samples < self.min_samples_split
    ):
        leaf_value = self._most_common_label(y)
        return Node(value=leaf_value)

    feat_idxs = np.random.choice(n_features, self.n_feats, replace=False)
    # greedily select the best split according to information gain
    best_feat, best_thresh = self._best_criteria(X, y, feat_idxs)
    # grow the children that result from the split
    left_idxs, right_idxs = self._split(X[:, best_feat], best_thresh)
    left = self._grow_tree(X[left_idxs, :], y[left_idxs], depth + 1)
    right = self._grow_tree(X[right_idxs, :], y[right_idxs], depth + 1)
    return Node(best_feat, best_thresh, left, right)

  def _best_criteria(self, X, y, feat_idxs):
    best_gain = -1
    split_idx, split_thresh = None, None
    for feat_idx in feat_idxs:
      X_column = X[:, feat_idx]
      thresholds = np.unique(X_column)
      for threshold in thresholds:
        gain = self._information_gain(y, X_column, threshold)
        if gain > best_gain:
          best_gain = gain
          split_idx = feat_idx
          split_thresh = threshold
    return split_idx, split_thresh

  def _information_gain(self, y, X_column, split_thresh):
    # parent loss
    parent_entropy = entropy(y)
    # generate split
    left_idxs, right_idxs = self._split(X_column, split_thresh)
    if len(left_idxs) == 0 or len(right_idxs) == 0:
      return 0
    # compute the weighted avg. of the loss for the children
    n = len(y)
    n_l, n_r = len(left_idxs), len(right_idxs)
    e_l, e_r = entropy(y[left_idxs]), entropy(y[right_idxs])
    child_entropy = (n_l / n) * e_l + (n_r / n) * e_r
    # information gain is difference in loss before vs. after split
    ig = parent_entropy - child_entropy
    return ig

  def _split(self, X_column, split_thresh):
    left_idxs = np.argwhere(X_column <= split_thresh).flatten()
    right_idxs = np.argwhere(X_column > split_thresh).flatten()
    return left_idxs, right_idxs

  def _traverse_tree(self, x, node):
    if node.is_leaf_node():
      return node.value

    if x[node.feature] <= node.threshold:
      return self._traverse_tree(x, node.left)
    return self._traverse_tree(x, node.right)

  def _most_common_label(self, y):
    counter = Counter(y)
    most_common = counter.most_common(1)[0][0]
    return most_common

  def accuracy(self, y_true, y_pred):
    return f"Accuracy: {np.equal(y_true, y_pred).sum()/len(y_true)*100}%"


### Testing the algorithm

We are going to load the dataset from `sklearn`.

In [10]:
from sklearn import datasets
from sklearn.model_selection import train_test_split

In [11]:
data = datasets.load_breast_cancer()
X, y = data.data, data.target

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [12]:
clf = DecisionTree(max_depth=10)
clf.fit(X_train, y_train)

In [13]:
y_pred = clf.predict(X_test)
acc = clf.accuracy(y_test, y_pred)
acc

'Accuracy: 94.73684210526315%'

### Ref

1. [geeks for geeks](https://www.geeksforgeeks.org/decision-tree-implementation-python/)
2. [python engineer](https://github.com/python-engineer/MLfromscratch/blob/master/mlfromscratch/decision_tree.py)